# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 11 2022 1:18PM,250613,19,ACG-2102491840,ACG North America LLC,Released
1,Nov 11 2022 1:16PM,250612,10,8758271,Eywa Pharma Inc.,Released
2,Nov 11 2022 1:16PM,250612,10,8758296,Eywa Pharma Inc.,Released
3,Nov 11 2022 1:16PM,250612,10,8758306,Eywa Pharma Inc.,Released
4,Nov 11 2022 1:16PM,250612,10,8759391,Eywa Pharma Inc.,Released
5,Nov 11 2022 1:16PM,250612,10,8759399,Eywa Pharma Inc.,Released
6,Nov 11 2022 1:16PM,250612,10,8759401,Eywa Pharma Inc.,Released
7,Nov 11 2022 1:16PM,250612,10,8754007,Eywa Pharma Inc.,Released
8,Nov 11 2022 1:16PM,250612,10,8754016,Eywa Pharma Inc.,Released
9,Nov 11 2022 1:16PM,250612,10,8750278-BO,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
35,250608,Released,13
36,250610,Released,1
37,250611,Released,1
38,250612,Released,9
39,250613,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
250608,NaN,NaN,13.0
250610,NaN,NaN,1.0
250611,NaN,NaN,1.0
250612,NaN,NaN,9.0
250613,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
250428,0.0,1.0,0.0
250518,0.0,0.0,5.0
250534,2.0,0.0,10.0
250545,0.0,0.0,1.0
250546,7.0,5.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
250428,0,1,0
250518,0,0,5
250534,2,0,10
250545,0,0,1
250546,7,5,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,250428,0,1,0
1,250518,0,0,5
2,250534,2,0,10
3,250545,0,0,1
4,250546,7,5,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,250428,,1,
1,250518,,,5
2,250534,2,,10
3,250545,,,1
4,250546,7,5,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 11 2022 1:18PM,250613,19,ACG North America LLC
1,Nov 11 2022 1:16PM,250612,10,Eywa Pharma Inc.
10,Nov 11 2022 1:15PM,250589,10,Eywa Pharma Inc.
11,Nov 11 2022 1:06PM,250611,21,"NBTY Global, Inc."
12,Nov 11 2022 12:59PM,250610,19,"GCH Granules USA, Inc."
13,Nov 11 2022 12:45PM,250608,10,ISDIN Corporation
26,Nov 11 2022 12:39PM,250606,10,ISDIN Corporation
30,Nov 11 2022 12:35PM,250604,15,Lonza
31,Nov 11 2022 12:19PM,250603,21,"NBTY Global, Inc."
32,Nov 11 2022 12:15PM,250602,19,"NAPP Technologies, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Nov 11 2022 1:18PM,250613,19,ACG North America LLC,,,1
1,Nov 11 2022 1:16PM,250612,10,Eywa Pharma Inc.,,,9
2,Nov 11 2022 1:15PM,250589,10,Eywa Pharma Inc.,,,1
3,Nov 11 2022 1:06PM,250611,21,"NBTY Global, Inc.",,,1
4,Nov 11 2022 12:59PM,250610,19,"GCH Granules USA, Inc.",,,1
5,Nov 11 2022 12:45PM,250608,10,ISDIN Corporation,,,13
6,Nov 11 2022 12:39PM,250606,10,ISDIN Corporation,,1,3
7,Nov 11 2022 12:35PM,250604,15,Lonza,,,1
8,Nov 11 2022 12:19PM,250603,21,"NBTY Global, Inc.",,,1
9,Nov 11 2022 12:15PM,250602,19,"NAPP Technologies, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 11 2022 1:18PM,250613,19,ACG North America LLC,1,,
1,Nov 11 2022 1:16PM,250612,10,Eywa Pharma Inc.,9,,
2,Nov 11 2022 1:15PM,250589,10,Eywa Pharma Inc.,1,,
3,Nov 11 2022 1:06PM,250611,21,"NBTY Global, Inc.",1,,
4,Nov 11 2022 12:59PM,250610,19,"GCH Granules USA, Inc.",1,,
5,Nov 11 2022 12:45PM,250608,10,ISDIN Corporation,13,,
6,Nov 11 2022 12:39PM,250606,10,ISDIN Corporation,3,1,
7,Nov 11 2022 12:35PM,250604,15,Lonza,1,,
8,Nov 11 2022 12:19PM,250603,21,"NBTY Global, Inc.",1,,
9,Nov 11 2022 12:15PM,250602,19,"NAPP Technologies, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 11 2022 1:18PM,250613,19,ACG North America LLC,1,,
1,Nov 11 2022 1:16PM,250612,10,Eywa Pharma Inc.,9,,
2,Nov 11 2022 1:15PM,250589,10,Eywa Pharma Inc.,1,,
3,Nov 11 2022 1:06PM,250611,21,"NBTY Global, Inc.",1,,
4,Nov 11 2022 12:59PM,250610,19,"GCH Granules USA, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 11 2022 1:18PM,250613,19,ACG North America LLC,1.0,NaN,NaN
1,Nov 11 2022 1:16PM,250612,10,Eywa Pharma Inc.,9.0,NaN,NaN
2,Nov 11 2022 1:15PM,250589,10,Eywa Pharma Inc.,1.0,NaN,NaN
3,Nov 11 2022 1:06PM,250611,21,"NBTY Global, Inc.",1.0,NaN,NaN
4,Nov 11 2022 12:59PM,250610,19,"GCH Granules USA, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 11 2022 1:18PM,250613,19,ACG North America LLC,1.0,0.0,0.0
1,Nov 11 2022 1:16PM,250612,10,Eywa Pharma Inc.,9.0,0.0,0.0
2,Nov 11 2022 1:15PM,250589,10,Eywa Pharma Inc.,1.0,0.0,0.0
3,Nov 11 2022 1:06PM,250611,21,"NBTY Global, Inc.",1.0,0.0,0.0
4,Nov 11 2022 12:59PM,250610,19,"GCH Granules USA, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4761028,90.0,8.0,9.0
15,751763,5.0,23.0,0.0
16,501019,0.0,3.0,0.0
19,1252998,5.0,0.0,0.0
21,1002313,4.0,0.0,0.0
22,250601,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4761028,90.0,8.0,9.0
1,15,751763,5.0,23.0,0.0
2,16,501019,0.0,3.0,0.0
3,19,1252998,5.0,0.0,0.0
4,21,1002313,4.0,0.0,0.0
5,22,250601,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,90.0,8.0,9.0
1,15,5.0,23.0,0.0
2,16,0.0,3.0,0.0
3,19,5.0,0.0,0.0
4,21,4.0,0.0,0.0
5,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,90.0
1,15,Released,5.0
2,16,Released,0.0
3,19,Released,5.0
4,21,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,21,22
Status,,,,,,
Completed,9.0,0.0,0.0,0.0,0.0,0.0
Executing,8.0,23.0,3.0,0.0,0.0,0.0
Released,90.0,5.0,0.0,5.0,4.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,21,22
0,Completed,9.0,0.0,0.0,0.0,0.0,0.0
1,Executing,8.0,23.0,3.0,0.0,0.0,0.0
2,Released,90.0,5.0,0.0,5.0,4.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,21,22
0,Completed,9.0,0.0,0.0,0.0,0.0,0.0
1,Executing,8.0,23.0,3.0,0.0,0.0,0.0
2,Released,90.0,5.0,0.0,5.0,4.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()